In [88]:
# load environment variables
from dotenv import load_dotenv
from pathlib import Path
import os

load_dotenv()

# selenium to parse dynamic web pages
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver import ChromeOptions
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

# import utilities
import pandas as pd
import time

# Authentication & Filters

In [89]:
# get environment variables
card_num = os.environ.get('MISS_LIB_CARD')
base_url = os.getenv('BASE_URL')

# search terms
query = "banquet"
data_values = [
    581212, # Banquet Facilities
    791102, # Ballrooms
    799904, # Auditoriums
    738931] # Convention & Meeting Facilities

In [90]:
print(base_url)

https://www.mississauga.ca/library/research-and-learn/business/


# Set Browser Options & Install Drivers

In [91]:
# define options for browser
options = ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
# options.add_argument('--headless')

# install driver and open URL
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# Define Helper Functions

In [92]:
def document_initialised(driver):
    '''
    document_initialised will command the driver to wait for the page to load completely.

    Args:
        driver (ChromeService object): Chrome browser driver.

    Returns:
        js script: Driver executes a script to wait for the page to load or, if it has already loaded before the listener can catch it, to proceed.
    '''
    return driver.execute_script("if (document.readyState === 'complete') {return true;} else {window.addEventListener('load', () => {return true;});}")

def wait_for_page(n):
    '''
    wait_for_page will stall the selenium driver to listen for conditions.

    Args:
        n (int): Set wait time in seconds.

    '''
    WebDriverWait(driver, timeout=10).until(document_initialised)
    time.sleep(n)
    return

def scrape_table_data(driver):
    '''
    scrape_page_data will scrape all data from the tabulated, high-level results.
    NOTE: This function only works if it is called on the first page of the tabulated results.

    Args:
        driver (WebDriver object): Chrome browser driver.

    Returns:
        data (list): Scraped entries from the tables.
        heading_names (list): Scraped fields from the tables.
    '''

    # find the resulting number of pages
    max_pages = driver.find_element(By.CLASS_NAME, "data-page-max").text

    # define containers
    data = list()
    heading_names = list()

    # scrape table headings
    headings = driver.find_elements(By.XPATH, "//thead[@id = 'searchResultsHeader']/tr/th")
    for heading in headings:
        heading_names.append(heading.text)

    for _ in range(1, int(max_pages)+1):
        # scrape entries from table
        rows = driver.find_elements(By.XPATH, "//tbody[@id = 'searchResultsPage']/tr")

        for row in rows:
            row_data = list()
            for col in row.find_elements(By.TAG_NAME, "td"):
                row_data.append(col.text)
            data.append(row_data)

        # go to the next page then wait
        driver.find_elements(By.CLASS_NAME, "next")[0].click()
        wait_for_page(3)

    return data, heading_names

def scrape_groupbox_info(groupbox, groupbox_title):
    '''
    scrape_groupbox_info will scrape groupbox details from one company page.

    Args:
        groupbox (WebDriver element): WebDriver element filtering for groupbox class divs.
        groupbox_title (WebDriver element): WebDriver element filtering for groupbox_title divs.

    Returns:
        temp_list (list): List of data values scraped from each groupbox.
        count (int): Length of items in temp_list.
    '''
    temp_list = list()
     #  confirm the case
    print(groupbox_title.text)

    # find all information in the groupbox
    cols = groupbox.find_elements(By.TAG_NAME, "td")
    count = len(cols)

    # append information to the info_list
    for col in cols:
        temp_list.append(col.text)

    # capture if website if present, return nothing if not
    # try:
    #     temp_list.append(groupbox.find_element(By.TAG_NAME, "a").get_attribute("href"))
    # except NoSuchElementException:
    #     temp_list.append("")

    return temp_list, count

# Navigate to URL to Results Table

In [93]:
# navigate to URL
driver.get(base_url)

# navigate to Canadian Businesses
wait_for_page(1)
driver.find_element(By.XPATH, ("//a[@title = 'Canadian Businesses']")).click()

# check off on the ToC and Agree
wait_for_page(1)
driver.find_element(By.ID, "chkAgree").click()
driver.find_element(By.CLASS_NAME, "action-agree").click()

# enter card number and log on
wait_for_page(1)
driver.find_element(By.ID, "matchcode").send_keys(card_num)
driver.find_element(By.CLASS_NAME, "action-submit-form").click()

# open Canadian Business search
wait_for_page(1)
driver.find_element(By.CLASS_NAME, ("action-do-search")).click()

# switch to Advanced Search and interact with filters
wait_for_page(1)
driver.find_element(By.CLASS_NAME, "advancedSearch").click()
driver.find_element(By.ID, "cs-YellowPageHeadingOrSic").click() # Business Type: Keyword/SIC/NAICS
driver.find_element(By.ID, "cs-Province").click() # Geography: Province

# wait for advanced filter options to populate
wait_for_page(1)
driver.find_element(By.CSS_SELECTOR, "input#VerifiedOnly").click()
driver.find_element(By.CSS_SELECTOR, "input#sicPrimaryOptionId").click() # radio: Search primary SIC Only
driver.find_element(By.ID, "sicLookupKeyword").send_keys(query) # search field: Query
driver.find_element(By.ID, "searchSic").click() # search button: Query

# wait for results to populate and highlight chosen query fields
wait_for_page(3)
for value in data_values:
    driver.find_element(By.XPATH, f"//li[@data-value = {value}]").click() # results: Highlight NAICS results
driver.find_element(By.XPATH, f"//li[@data-value = 'ON']").click() # results: Highlight Ontario results
driver.find_element(By.CLASS_NAME, "action-view-results").click() # View Results with filters

# results page
wait_for_page(3)
driver.find_element(By.XPATH, "//*[@alt = 'Optional Column']").click()
driver.find_element(By.XPATH, "//*[@data-key = 'Title']").click()

### Make function to repeat try/except blocks for each check below.

In [94]:
# navigate to the first company
## LOOP THROUGH RESULTS
wait_for_page(1)
company_links = driver.find_elements(By.XPATH, "//tbody[@id ='searchResultsPage']/tr/td/a")
company_links[0].click()


In [95]:
info_list = list()

# find group boxes
groupbox_titles = driver.find_elements(By.CLASS_NAME, "groupboxTitle")

In [96]:
# Isolate site for each group box based on title
for i, groupbox_title in enumerate(groupbox_titles):
    # Save parent element of group box titles
    groupboxes = driver.find_elements(By.CLASS_NAME, "groupbox")

    match groupbox_title.text:
        case "Location Information":
            temp_list, count = scrape_groupbox_info(groupboxes[i], groupbox_title)
            info_list.extend(temp_list)
            if count < 10:
                info_list.append("")

        case "Industry Profile":
            temp_list, _ = scrape_groupbox_info(groupboxes[i], groupbox_title)
            info_list.extend(temp_list)

        case  "Photo, Map, & Directions":
            temp_list, _ = scrape_groupbox_info(groupboxes[i], groupbox_title)
            info_list.extend(temp_list)

        case "Business Demographics":
            temp_list, _ = scrape_groupbox_info(groupboxes[i], groupbox_title)
            info_list.extend(temp_list)

        case "Management Directory":
            temp_list, _ = scrape_groupbox_info(groupboxes[i], groupbox_title)
            info_list.extend(temp_list)

        case "Business Expenditures":
            temp_list, _ = scrape_groupbox_info(groupboxes[i], groupbox_title)
            info_list.extend(temp_list)

        case "Nearby Businesses":
            temp_list, _ = scrape_groupbox_info(groupboxes[i], groupbox_title)
            info_list.extend(temp_list)

        case "Competitors Report":
            temp_list, _ = scrape_groupbox_info(groupboxes[i], groupbox_title)
            info_list.extend(temp_list)

        case _:
            print("")

Location Information
Industry Profile
Photo, Map, & Directions
Business Demographics
Management Directory
Business Expenditures
Nearby Businesses
Competitors Report


In [97]:
print(len(info_list))
info_list

48


['10 Tation Event Catering',
 '',
 '232 Norseman St',
 'Etobicoke, ON M8Z2R4',
 '',
 '(416) 243-5144',
 'Toronto Metropolitan',
 '(416) 243-2662',
 'Toronto',
 '10tation.com',
 '5812-12',
 'Caterers',
 '',
 'Regular',
 '2019',
 '72232001',
 'Caterers',
 '',
 'Regular',
 '2019',
 '24',
 '$1,296,000',
 'Not Available',
 'Not Available',
 'Single Loc',
 'Excellent',
 '',
 '43.632250 / -79.529510',
 '03-839-5380',
 'No',
 "David D'Aprile",
 'Owner',
 'Male',
 'These expenditures are an estimated annual expense.',
 '$2,500 to $5,000',
 '$20,000 to $50,000',
 '$1,000 to $10,000',
 '$10,000 to $25,000',
 '$1,000 to $2,500',
 '$5,000 to $10,000',
 '$20,000 to $50,000',
 '$10,000 to $25,000',
 '$250,000 to $500,000',
 '$1,000 to $2,500',
 '$50,000 to $100,000',
 '$1,000 to $2,500',
 '$2,000 to $5,000',
 '$25,000 to $50,000']

In [98]:
driver.back()

In [99]:
# df = pd.DataFrame(data, columns=headings)

In [100]:
# df.to_excel('lead-list-scraped.xlsx')

In [101]:
#Location Information
## Company
## Address
## Phone
## Fax
## District
## CMA

#Industry Profile
## SIC Code
## Descriptions
## Ad Size
## Year First Appeared
## NAICS Code
## Description
## Ad Size
## Year First Appeared

# Business Demographics
## Location Employees
## Corporate Employees
## Location Type
## Stock Ticker Symbol
## IUSA Number
## Location Sales Volume
## Corporate Sales Volume
## Credit Rating
## Latitude / Longitude
## Federal Contractor

# Management Directory <-- Use this to populate your rows
## Name
## Executive Title
## Gender

# Business Expenditures
## Accounting
## Contract Labor
## Legal
## Office Equipment & Supplies
## Payroll & Benefits
## Rent & Leasing
## Telecommunications
## Advertising
## Insurance
## Management/Administration
## Package/Container
## Purchased Print
## Technology
## Utilities